<a href="https://colab.research.google.com/github/usma11dia0/web_scraping_on_colab/blob/main/web_scraping_on_colab_tabelog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.oCNKK7fV3e/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.2PjztGbXAQ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.TeZPjbWXmg/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [ ]:
#環境変数設定
TARGET_URL = 'https://tabelog.com/saitama/'
NUM_TO_FETCH = 40
STORES_PER_PAGE = 20
DRIVER_WAIT_TIME = 2

# 各種ファイル保存先
ACCU_LIST_PATH = '/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/accumulated_list_tabelog.csv'
SUBMIT_FILE_PATH ='/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/accumulated_list_tabelog.xlsx'

#取得データ格納先
result_dict = { 
    '店名': [],
    '電話番号':[],
    '住所': [],
    'URL':[],
}

In [ ]:
#標準ライブラリ
import os
import math
import json

#サードパーティライブラリ
import pandas as pd
from google.colab import drive
from selenium import webdriver
from selenium.webdriver.common.by import By
from logging import (
    getLogger, 
    StreamHandler, 
    DEBUG, 
    INFO, 
    Formatter, 
    config
)
# drive.mount('/content/drive')

In [ ]:
#関数定義
def add_store_url(elements, store_urls, count_skip):
    for a_element in a_elements:
        store_url = a_element.get_attribute('href')
        if store_url not in accu_set:
            store_urls.append(store_url)
        else:
            count_skip += 1
    return store_urls, count_skip

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument("--disable-extensions")
options.add_argument("--disable-popup-blocking")
options.add_argument(
       "user-agent=Mozilla/5.0 (X11; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0"
    )
options.add_experimental_option("prefs", {
    "profile.managed_default_content_settings.images": 2,  # 画像の無効化
    "profile.managed_default_content_settings.plugins": 2,  # プラグインの無効化
})

driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(DRIVER_WAIT_TIME)

In [ ]:
target = '埼玉県'

#カスタムロガーの設定
with open('/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/logging_config.json', 'r') as f:
    logger_config = json.load(f)
config.dictConfig(logger_config)
logger = getLogger('main')

try:
      driver.get(TARGET_URL)
      logger.debug(f'{target}：トップページへ移動しました')
except Exception as e:
      logger.error(f'{target}：トップページへ移動出来ませんでした: [e]')
      raise

2023-04-14 16:23:13,238 [DEBUG] main: 埼玉県：トップページへ移動しました


In [ ]:
#店舗リストの全ページ数を導出
span_elements = driver.find_elements(By.CSS_SELECTOR,".c-page-count__num")
num_all_stores = int(span_elements[-1].text)
num_all_pages = math.ceil(num_all_stores / STORES_PER_PAGE)

#積み上げリストから既存取得の店舗URLを抽出
df_accu = pd.read_csv(ACCU_LIST_PATH, usecols=[3])
accu_list = df_accu.iloc[:, 0].values.tolist()
accu_set = set(accu_list)

# 目標件数を満たすまでstore_urlを取得
# 1ページ目
try:
  a_elements = driver.find_elements(By.CSS_SELECTOR,".list-rst__rst-name a")
  store_urls = []
  count_skip = 0
  store_urls, count_skip = add_store_url(a_elements, store_urls, count_skip)
  logger.debug(
    f'{target}：店舗リスト取得 (取得数:{len(store_urls)}, 重複数：{count_skip})'
    )
except  Exception as e:
  logger.error(f'{target}：店舗リストへ移動出来ませんでした: [e]')
  raise

#2ページ目以降
i = 1
while len(store_urls) < NUM_TO_FETCH:
  i += 1
  page_url = f'https://tabelog.com/saitama/rstLst/{i}/'
  try:
    if i < num_all_pages:
      count_skip = 0
      driver.get(page_url)
      a_elements = driver.find_elements(By.CSS_SELECTOR,".list-rst__rst-name a")
      store_urls, count_skip = add_store_url(a_elements, store_urls, count_skip)
      logger.debug(
        f'{target}：店舗リスト{i}取得 (累計:{len(store_urls)}, 重複：{count_skip})'
      )
    else:
      logger.info(f'{target}：取得出来る店舗情報がありません')
  except Exception as e:
    logger.error(f'{target}：店舗リスト{i}へ移動出来ませんでした: [e]')

logger.info(f'{target}：店舗URL取得完了:{len(store_urls)}個')  


2023-04-14 16:26:36,016 [DEBUG] main: 埼玉県：店舗リスト取得 (取得数:20, 重複数：0)
2023-04-14 16:27:10,307 [DEBUG] main: 埼玉県：店舗リスト2取得 (累計:20, 重複：20)
2023-04-14 16:29:30,588 [DEBUG] main: 埼玉県：店舗リスト3取得 (累計:39, 重複：1)
2023-04-14 16:29:38,612 [DEBUG] main: 埼玉県：店舗リスト4取得 (累計:59, 重複：0)
2023-04-14 16:29:38,614 [INFO] main: 埼玉県：店舗URL取得完了:59個


In [ ]:
#店舗個別ページにて情報取得
page_num = 0
for store_url in store_urls:
  try:
      driver.get(store_url)
      page_num += 1
      logger.debug(f'{target}：店舗ページ{page_num}へ移動しました')
  except Exception as e:
      logger.error(f'{target}：店舗ページへ移動出来ませんでした: [e]')

  try:
      #店舗名/住所/電話番号取得
      store_name = driver.find_element(By.CSS_SELECTOR,".display-name").text
      phone_number = driver.find_element(By.CSS_SELECTOR,".rstinfo-table__tel-num").text 
      address = driver.find_element(By.CSS_SELECTOR,".rstinfo-table__address").text
      logger.debug(f'{target}：店舗ページ{page_num}の取得完了')
      #結果格納
      result_dict['店名'].append(store_name)
      result_dict['電話番号'].append(phone_number)
      result_dict['住所'].append(address)
      result_dict['URL'].append(store_url)
  except Exception as e:
      logger.error(f'{target}：{store_name}の店舗情報取得に失敗しました: [e]')

df_result = pd.DataFrame(result_dict)
df_result

2023-04-14 15:10:37,693 [DEBUG] main: 埼玉県：店舗ページ2へ移動しました
2023-04-14 15:10:37,856 [DEBUG] main: 埼玉県：店舗ページ2の取得完了
2023-04-14 15:10:41,044 [DEBUG] main: 埼玉県：店舗ページ3へ移動しました
2023-04-14 15:10:41,198 [DEBUG] main: 埼玉県：店舗ページ3の取得完了
2023-04-14 15:10:43,913 [DEBUG] main: 埼玉県：店舗ページ4へ移動しました
2023-04-14 15:10:44,047 [DEBUG] main: 埼玉県：店舗ページ4の取得完了
2023-04-14 15:10:46,567 [DEBUG] main: 埼玉県：店舗ページ5へ移動しました
2023-04-14 15:10:46,699 [DEBUG] main: 埼玉県：店舗ページ5の取得完了
2023-04-14 15:10:48,836 [DEBUG] main: 埼玉県：店舗ページ6へ移動しました
2023-04-14 15:10:49,018 [DEBUG] main: 埼玉県：店舗ページ6の取得完了
2023-04-14 15:10:51,758 [DEBUG] main: 埼玉県：店舗ページ7へ移動しました
2023-04-14 15:10:51,881 [DEBUG] main: 埼玉県：店舗ページ7の取得完了
2023-04-14 15:10:54,157 [DEBUG] main: 埼玉県：店舗ページ8へ移動しました
2023-04-14 15:10:54,267 [DEBUG] main: 埼玉県：店舗ページ8の取得完了
2023-04-14 15:10:56,239 [DEBUG] main: 埼玉県：店舗ページ9へ移動しました
2023-04-14 15:10:56,334 [DEBUG] main: 埼玉県：店舗ページ9の取得完了
2023-04-14 15:10:58,780 [DEBUG] main: 埼玉県：店舗ページ10へ移動しました
2023-04-14 15:10:58,865 [DEBUG] main: 埼玉県：店舗ページ10の取得完了
2023-04-

,店名,電話番号,住所,URL
0,ガズル,050-5869-4620,埼玉県朝霞市青葉台1-3-5,https://tabelog.com/saitama/A1103/A110301/1103...
1,KICHIRI 所沢,050-5590-1696,埼玉県所沢市日吉町8-4 マスダ写真館ビル4F,https://tabelog.com/saitama/A1106/A110601/1104...
2,カンパイ酒場,050-5590-3065,埼玉県さいたま市大宮区宮町1-97-2 大宮二葉ビル3 2F,https://tabelog.com/saitama/A1101/A110101/1105...
3,ひねり蛇口ハイ 大衆酒泉テルマエ 所沢泉,050-5890-7121,埼玉県所沢市日吉町8-3 三上ビル 2F,https://tabelog.com/saitama/A1106/A110601/1105...
4,個室居酒屋 甚平 南越谷店,050-5869-0705,埼玉県越谷市南越谷1-12-12 東京宝石本社ビル 3F,https://tabelog.com/saitama/A1102/A110203/1104...
5,エビス朝霞 総本店,050-5589-2090,埼玉県朝霞市仲町1-11-48 蕪木ビル 1F,https://tabelog.com/saitama/A1103/A110301/1103...
6,2000円 食べ放題飲み放題 居酒屋 おすすめ屋 大宮店,050-5456-8568,埼玉県さいたま市大宮区仲町1-111-2 マルハチビル 3F,https://tabelog.com/saitama/A1101/A110101/1104...
7,ステーキのどん 千間台店,050-5589-9374,埼玉県越谷市千間台東1-16-2,https://tabelog.com/saitama/A1102/A110203/1102...
8,個室完備 大人の隠れ家個室 奏屋 南越谷店,050-5457-2142,埼玉県越谷市南越谷2-2-12 コアーズビル 2F,https://tabelog.com/saitama/A1102/A110203/1104...
9,WILLIAMS BROOKLYN RESTAURANT,050-5456-8631,埼玉県久喜市久喜中央1-3-14,https://tabelog.com/saitama/A1104/A110402/1104...


In [ ]:
# 提出ファイル用出力
df_result.to_excel(SUBMIT_FILE_PATH, header=True, index=False)

# 積み上げリスト保存用出力
df_result.to_csv(ACCU_LIST_PATH, mode='a', header=True, index=False)

In [ ]:
# 積み上げリスト確認
df_accu = pd.read_csv(ACCU_LIST_PATH)
df_accu

,店名,電話番号,住所,URL
0,広島鉄板 毛利,050-5595-9128,埼玉県富士見市ふじみ野東1-21-1 ラガールふじみ野102,https://tabelog.com/saitama/A1103/A110302/1102...
1,完全個室焼肉 最高ランク雌牛専門店 OBORO 大宮店,050-5571-4375,埼玉県さいたま市大宮区宮町1-99-3 L.H大宮ビル 3F,https://tabelog.com/saitama/A1101/A110101/1105...
2,だるま屋 大宮本店,050-5890-3634,埼玉県さいたま市大宮区仲町1-33-4 いつしビル 2F,https://tabelog.com/saitama/A1101/A110101/1105...
3,炉端ト酒 舌心,050-5457-0832,埼玉県新座市東北1-14-1 志幸30SEASON 1F,https://tabelog.com/saitama/A1103/A110302/1105...
4,個室居酒屋 美味門 大宮店,050-5597-7141,埼玉県さいたま市大宮区桜木町1-8-3 サンユービル 5Ｆ,https://tabelog.com/saitama/A1101/A110101/1104...
...,...,...,...,...
56,焼肉 ニクラボ 川越駅前店,050-5457-0772,埼玉県川越市菅原町22-2 第二MSビル 4F,https://tabelog.com/saitama/A1103/A110303/1105...
57,個室ダイニング ウメ子の家 大宮東口駅前店,050-5869-5852,埼玉県さいたま市大宮区大門町1-61-1 DK大宮ビル6階,https://tabelog.com/saitama/A1101/A110101/1101...
58,寿司・懐石處 やなぎ,050-5595-4667,埼玉県飯能市仲町2-10,https://tabelog.com/saitama/A1106/A110603/1100...
59,じろべ 大宮店,050-5890-2033,埼玉県さいたま市大宮区宮町1-41 八千代ビル ２Ｆ,https://tabelog.com/saitama/A1101/A110101/1100...
